# 1. Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import pandas as pd

# 2. Get HTML Content

In [2]:
#import requests
url = 'https://www.habitaclia.com/viviendas-madrid.htm'
html = requests.get(url).content
html[:500]

b'\r\n<!DOCTYPE html>\r\n<html lang="es">\r\n\r\n<head xmlns="http://www.w3.org/1999/xhtml" lang="es">\r\n    <!-- id srv=HABIT-WEB-07 -->\r\n    <title>Viviendas en Madrid - habitaclia</title>\r\n    <meta name="description" content="14.042 viviendas en venta en Madrid de particulares, agencias inmobiliarias y bancos. Encuentra tambi\xc3\xa9n viviendas en alquiler y viviendas obra nueva en Madrid. Para buscar viviendas en venta en Madrid, habitaclia, cuando te ayudan, es m\xc3\xa1s f\xc3\xa1cil." />\r\n    <meta name="Author" con'

In [3]:
#from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [4]:
#soup

In [5]:
# The following part is added simply to remember that there may be issues with BS4 auto detection of a page encoding.
# A "from_encoding" parameter can be used to specify encoding.
# A more detailed explanation can be found in this link: (Also contains Beautiful Soup Tutorial)
#https://www.tutorialspoint.com/beautiful_soup/beautiful_soup_encoding.htm

soup2 = BeautifulSoup(html,'html.parser',from_encoding='utf-8')

Resource:
https://www.tutorialspoint.com/beautiful_soup/beautiful_soup_encoding.htm

In [6]:
soup == soup2

# I was trying to check if both outputs are exactly the same. But I'm not sure if its just confirming that both are bs4.BeautifulSoup objects.

True

In [7]:
type(soup)

bs4.BeautifulSoup

In [8]:
# Running an HTML document through Beautiful Soup gives us a BeautifulSoup object, which represents the document as a nested data structure:
# Which can be checked using soup.pretify()
#print(soup.prettify())

In [9]:
# Then, if we just call the tags we can see whats contained in each different tag.
# Which means it becomes useful to see all the tags in our HTML Document. See below

# soup.title, soup.text

# 3. Exploring Tags

In [10]:
tag_list = []
tag_set = set()

for tag in soup.find_all(True):
    #print(tag.name)
    if tag.name not in tag_list:
        tag_list.append(tag.name)
    if tag.name not in tag_set:
        tag_set.add(tag.name)
        
print(tag_list, '\nThis list lenght is:', len(tag_list))
print()
print(tag_set, '\nThis set lenght is:', len(tag_set))

['html', 'head', 'title', 'meta', 'link', 'script', 'style', 'body', 'noscript', 'iframe', 'header', 'div', 'a', 'svg', 'use', 'ul', 'li', 'span', 'img', 'map', 'nav', 'h1', 'aside', 'button', 'p', 'strong', 'form', 'input', 'label', 'main', 'section', 'select', 'option', 'h2', 'article', 'h3', 'sup', 'textarea', 'footer'] 
This list lenght is: 39

{'section', 'header', 'aside', 'span', 'title', 'svg', 'noscript', 'map', 'p', 'select', 'use', 'li', 'strong', 'link', 'html', 'textarea', 'img', 'a', 'script', 'div', 'h1', 'main', 'article', 'button', 'label', 'style', 'meta', 'head', 'option', 'nav', 'iframe', 'body', 'h3', 'ul', 'footer', 'sup', 'input', 'h2', 'form'} 
This set lenght is: 39


Using inspect on the browser and exploring options I found that the tag "h3" was the most useful to get the unique listing links.

I believe this is important because I need to iterate or go to each link to retrieve all the relevant info from each listing.

I left the "queries" to document and remember my thought process and the actual process of getting the data.

In [11]:
# Using soup.find all to explore tags and/or classes.

raw_listings_titles = [x.text for x in soup.find_all('h3', {'class': 'list-item-title'})]

raw_listings_titles

['\nPiso  en  Arapiles. Fenomenal\n',
 '\nPiso  en  Gaztambide. Con terraza\n',
 '\nPiso  en  C/ doctor juan jose lopez 0\n',
 '\nPiso  en  Universidad-Malasaña. Espléndido\n',
 '\nPiso  en  Gaztambide. Estupendo\n',
 '\nPiso  en  Palos de Moguer. Genial\n',
 '\nPiso  en  Las Tablas. Estupendo\n',
 '\nPiso  en  Lista. Admirable\n',
 '\nPiso  en  Embajadores-Lavapiés. Impecable piso en lavapies - reforma integral\n',
 '\nPiso  en  Calle de modesto lafuente 46. Piso en venta en calle de modesto lafuente 46\n',
 '\nPiso  en  Canillas. Acogedor\n',
 '\nPlanta baja  en  Las Águilas. Fenomenal\n',
 '\nPiso  en  San Diego. Espléndido\n',
 '\nÁtico  en  Universidad-Malasaña. Ático con 95 m² de terraza en venta en sol, madrid\n',
 '\nPiso  en  Arroyo del Fresno. Con certificado passivhaus\n',
 '\nDuque de Rivas vuelta con Colegiata\n']

In [12]:
# Using soup.find all to explore tags and/or classes.

raw_listings_list_items = [x.text for x in soup.find_all('section', {'class': 'list-items'})]

# raw_listings_list_items

In [13]:
# Alternate way to find and print tags or specific strings.

#This option was to broad.
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [14]:
# Looking a little bit better, but annoyed by the None's in the output.

for link in soup.find_all('article')[0:5]:
        print(link.get('data-href'))

https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55
None
https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&geo=p&from=list&lo=55
None
https://www.habitaclia.com/comprar-piso-c_doctor_juan_jose_lopez_0_valdezarza-madrid-i23878000004181.htm?f=&geo=p&from=list&lo=55


In [15]:
# This tag seemed like a winner, but realized later that it was giving me one unwanted advertized/promo listing.

# I was getting 16 elements and looks like I need to restrict the table to get only 15 elements.

# Which are the elements that are truly real listings.

link_soup_table = soup.find_all('h3', {'class': 'list-item-title'})

link_soup_table[0:4]

[<h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Arapiles. Fenomenal">Piso  en  Arapiles. Fenomenal</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Gaztambide. Con terraza">Piso  en  Gaztambide. Con terraza</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-c_doctor_juan_jose_lopez_0_valdezarza-madrid-i23878000004181.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  C/ doctor juan jose lopez 0">Piso  en  C/ doctor juan jose lopez 0</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-esplendido_universidad_malasana-madrid-i15511003

In [16]:
soup.h3

<h3 class="list-item-title">
<a href="https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Arapiles. Fenomenal">Piso  en  Arapiles. Fenomenal</a>
</h3>

In [17]:
soup.h3.a

<a href="https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Arapiles. Fenomenal">Piso  en  Arapiles. Fenomenal</a>

In [18]:
soup.h3.a['href']

'https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55'

In [19]:
soup.h3.a['href'].startswith("https://www.habitaclia.com/comprar", 0, 34)

True

In [20]:
len('https://www.habitaclia.com/comprar')

34

In [21]:
len('https://www.habitaclia.com/fv')

29

In [22]:
soup.h3.a['href'].startswith("https://www.habitaclia.com/fv", 0, 29)

False

In [23]:
link_soup_table[0:5]

[<h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Arapiles. Fenomenal">Piso  en  Arapiles. Fenomenal</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Gaztambide. Con terraza">Piso  en  Gaztambide. Con terraza</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-c_doctor_juan_jose_lopez_0_valdezarza-madrid-i23878000004181.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  C/ doctor juan jose lopez 0">Piso  en  C/ doctor juan jose lopez 0</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-esplendido_universidad_malasana-madrid-i15511003

In [24]:
link_soup_test = soup.find_all('section', {'class': 'list-items'})

unique_link = link_soup_test[0].find_all('h3', {'class': 'list-item-title'})

unique_link[0].a['href']

'https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55'

In [25]:
link_soup_table[0].a['href']

'https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55'

In [26]:
link_soup_table[0]

<h3 class="list-item-title">
<a href="https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Arapiles. Fenomenal">Piso  en  Arapiles. Fenomenal</a>
</h3>

In [27]:
link_soup_table[1]

<h3 class="list-item-title">
<a href="https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Gaztambide. Con terraza">Piso  en  Gaztambide. Con terraza</a>
</h3>

In [28]:
link_soup_table[0].a['href']

'https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55'

In [29]:
link_soup_table[1].a['href']

'https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&geo=p&from=list&lo=55'

In [30]:
len(link_soup_table)

16

In [31]:
len(unique_link)

15

In [32]:
unique_link

[<h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Arapiles. Fenomenal">Piso  en  Arapiles. Fenomenal</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  Gaztambide. Con terraza">Piso  en  Gaztambide. Con terraza</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-c_doctor_juan_jose_lopez_0_valdezarza-madrid-i23878000004181.htm?f=&amp;geo=p&amp;from=list&amp;lo=55" itemprop="name" target="_self" title="Piso  en  C/ doctor juan jose lopez 0">Piso  en  C/ doctor juan jose lopez 0</a>
 </h3>,
 <h3 class="list-item-title">
 <a href="https://www.habitaclia.com/comprar-piso-esplendido_universidad_malasana-madrid-i15511003

In [33]:
data = []

for i in range(len(unique_link)):
    print(unique_link[i].a['href'])
    data.append(unique_link[i].a['href'])

https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-c_doctor_juan_jose_lopez_0_valdezarza-madrid-i23878000004181.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-esplendido_universidad_malasana-madrid-i15511003002573.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-estupendo_gaztambide-madrid-i15511003002757.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-genial_palos_de_moguer-madrid-i15511003002586.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-estupendo_las_tablas-madrid-i15511003002656.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-admirable_lista-madrid-i15511003002839.htm?f=&geo=p&from=list&lo=55
https://www.habitaclia.com/comprar-piso-impecable_en_lavapies_reforma_i

In [34]:
len(data)

15

In [35]:
# import libraries
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

###########################
# Loop of all the pages
###########################

#Loop to go over all pages
pages = np.arange(1, 5000, 1)
#data=[]

for page in tqdm(pages[:20]):
    try:
        page="https://www.habitaclia.com/viviendas-madrid-" + str(page) + ".htm"
        html = requests.get(page).content
        soup = BeautifulSoup(html, 'html.parser')
        link_soup_table = soup.find_all('section', {'class': 'list-items'})
        unique_link = link_soup_table[0].find_all('h3', {'class': 'list-item-title'})
        for i in range(len(unique_link)):
            data.append(unique_link[i].a['href'])
      
    except Exception as e:
        print("Oops!", print(e), "occurred.")
        print()
        break

100%|██████████| 20/20 [00:18<00:00,  1.06it/s]


In [36]:
len(unique_link)

15

In [37]:
len(data)

315

In [38]:
data[0:20]

['https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-piso-con_terraza_gaztambide-madrid-i15511003002590.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-piso-c_doctor_juan_jose_lopez_0_valdezarza-madrid-i23878000004181.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-piso-esplendido_universidad_malasana-madrid-i15511003002573.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-piso-estupendo_gaztambide-madrid-i15511003002757.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-piso-genial_palos_de_moguer-madrid-i15511003002586.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-piso-estupendo_las_tablas-madrid-i15511003002656.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-piso-admirable_lista-madrid-i15511003002839.htm?f=&geo=p&from=list&lo=55',
 'https://www.habitaclia.com/comprar-pi

In [39]:
data[0]

'https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55'

In [40]:
type(data[0])

str

In [41]:
#This was done to check my first link on the data list and retrieve price.

link = data[8]
html = requests.get(link).content
unique_soup = BeautifulSoup(html, 'html.parser')
summary_box_table = unique_soup.find_all('div', {'class': 'summary-left'})
tag_div_class_price = summary_box_table[0].find_all('div', {'class': 'price'})
zooming_at_price = tag_div_class_price[0].find_all('span', {'class': 'font-2'})
zooming_at_price[0].text
price = zooming_at_price[0].text[:-2]
price

'290.000'

In [42]:
#This was done to check my first link on the data list and retrieve the listing description.

listing_description = summary_box_table[0].find_all('h1')
description = listing_description[0].text
description

'Piso impecable piso en lavapies - reforma integral en Madrid'

In [43]:
#This was done to check my first link on the data list and retrieve the pinned location.

zooming_location = summary_box_table[0].find_all('article', {'class': 'location'})
pinned_location = zooming_location[0].find_all('a', {'class': 'jqVerMapaZonaTooltip link-map-location'})
location = pinned_location[0].get('title')
location_name = location[4:]
location_name

'Embajadores-Lavapiés'

In [45]:
# Listing summarized details in the feature container. => Square meters, # of Rooms, # of Bathrooms, Price per square meter

feature_container = summary_box_table[0].find_all('ul', {'class': 'feature-container'})

meters = []
rooms = []
baths = []
euros_per_meter = []

meters_pos_in_feature_container = feature_container[0].text.find('m2')
room_pos_in_feature_container = feature_container[0].text.find('hab.')
baths_pos_in_feature_container = feature_container[0].text.find('baño')
euro_pos_in_feature_container = feature_container[0].text.find('€/m')


if str('m2') in feature_container[0].text.strip():
    meters.append(feature_container[0].text[:feature_container[0].text.find('m2')].strip())
else:
    meters.append(np.NaN)

if str('hab') in feature_container[0].text:
    rooms.append(feature_container[0].text[int(feature_container[0].text.find('hab'))-4:int(feature_container[0].text.find('hab'))].strip())
else:
    rooms.append(np.NaN)

if str('baño') in feature_container[0].text:
    baths.append(feature_container[0].text[int(feature_container[0].text.find('baño'))-3:int(feature_container[0].text.find('baño'))].strip())
else:
    baths.append(np.NaN)
    
if str('€/m') in feature_container[0].text:
    euros_per_meter.append(feature_container[0].text[int(feature_container[0].text.find('€/m'))-8:int(feature_container[0].text.find('€/m'))].strip())
else:
    euros_per_meter.append(np.NaN)


#print(meters_pos_in_feature_container)
#print(room_pos_in_feature_container)
#print(baths_pos_in_feature_container)
#print(euro_pos_in_feature_container)

print('meters:', meters)
print('rooms:', rooms)
print('baths:', baths)
print('euros:', euros_per_meter)



meters: ['70']
rooms: ['2']
baths: ['1']
euros: ['4.143']


In [46]:
#LO QUE ESTUVE REVISANDO CON OCTAVIO

# import re

# feature_list = []

# for i in range(len(features)):
#     key = re.findall('\d+', features[i].text)[0]
#     value = re.findall('\d+', features[i].text)[0]
#     feature_list.append((key,value))
# print(feature_list)

In [47]:
#This was done to check my first link on the data list and retrieve the main characteristics.

# feature_container = summary_box_table[0].find_all('ul', {'class': 'feature-container'})
# features = summary_box_table[0].find_all('li', {'class': 'feature'})
# #print(features[0].text.strip('\n')) #THIS WAS THE CODE BASED ON FEATURE INDEX POSITION 0 => m2, 1 => #_of_rooms, 2 => bathrooms, 3 => price_per_square_meter
# #print(features[1].text.strip())
# #print(features[2].text.strip().rstrip('\r\n'))
# #print(features[3].text)

# # squared_meters = []
# # number_of_rooms = []
# # number_of_bathrooms = []
# # price_per_squared_meter = []


# for i in range(len(features)):

#     try:
#         squared_meters.append(int(features[0].text.strip('\ndesde ').strip('\n')[:-3]))
        
#     except:
#         squared_meters.append(np.NaN)

#     # TO EXTRACT NUMBER OF ROOMS
    
#     all_features = []
#     if str('hab.') in features[i].text:
#         try:
#             print('CON hab. en el string y deberia extraer rooms')
#             #room_pos_in_features = features[i].text.find('hab.')
#             number_of_rooms.append(int(features[i].text.strip()[:-5]))
#         except:
#             number_of_rooms.append(np.NaN)
#             print('CON hab. en el string y deberia definir rooms con NAN')
            
#     else:
#         number_of_rooms.append(np.NaN)
        
#         print('SIN hab. en el string y deberia definir rooms con NAN SOLO si NO esta definido aun')
#         number_of_rooms.append(np.NaN)

#     # TO EXTRACT NUMBER OF BATHROOMS
    
#     if str('baño') in features[i].text:
#         try:
#             #bathroom_pos_in_features = features[i].text.find('baño')
#             number_of_bathrooms = features[i].text.strip().rstrip('\r\n')
#             number_of_bathrooms = int(features[i].text.strip().rstrip('\r\n')[0:2].strip())
#                                    except:
#             number_of_bathrooms = np.NaN
#             pass
#             #number_of_bathrooms = np.NaN
#     else:
#         pass   

#     # TO EXTRACT NUMBER OF BATHROOMS

#     if str('€/m') in features[i].text:
#         try:
#             #price_squared_meters_pos = features[i].text.find('€/m')
#             price_per_squared_meter = features[i].text
#             price_per_squared_meter = features[i].text[:-5]
#         except:
#             price_per_squared_meter = np.NaN
#     else:
#         pass

# print('squared_meters:', squared_meters)
# print('number_of_rooms:', number_of_rooms)
# print('number_of_bathrooms:', number_of_bathrooms)
# print('price_per_squared_meter:', price_per_squared_meter)

In [48]:
lista_sample = ['prueba', 'hab.', 'rooms', 'baths']

str('hab.') in lista_sample

True

In [49]:
#This was done to try to find and narrow down on a specific feature/position.

feature_container = summary_box_table[0].find_all('ul', {'class': 'feature-container'})
features = summary_box_table[0].find_all('li', {'class': 'feature'})
str('hab') in features[1].text

True

In [50]:
# First try to retrieve "caracteristicas principales"

# for i in range(len(features)):
#     #print(features[i].text.find('baño'))
#     bathroom_pos_in_features = features[i].text.find('baño')
#     number_of_bathrooms = features[bathroom_pos_in_features].text.strip().rstrip('\r\n')
#     number_of_bathrooms = int(features[bathroom_pos_in_features].text.strip().rstrip('\r\n')[0:2].strip())
#     #number_of_bathrooms_list.append(number_of_bathrooms)
    
    
#     #print(features[i].text.find(' hab.'))
#     room_pos_in_features = features[i].text.find('hab.')
#     number_of_rooms = features[1].text.strip()
#     number_of_rooms = int(features[1].text.strip()[:-5])
#     #number_of_rooms_list.append(number_of_rooms)
    
    
#     #print(features[i].text.find('€/m'))
#     price_squared_meters_pos = features[i].text.find('€/m')
#     price_per_squared_meter = features[3].text
#     price_per_squared_meter = features[3].text[:-5]
    
    
# print('number_of_baths:', number_of_bathrooms)
# print('number_of_rooms:', number_of_rooms)
# print('price_per_squared_meter:', price_per_squared_meter)

In [51]:
#This was done to check my first link on the data list and retrieve the detailed description.

description_table = unique_soup.find_all('section', {'class': 'detail'})
detailed_description = description_table[0].find_all('p', {'class': 'detail-description'})
detailed_description[0].text.replace('\xa0', ' ').replace('  ', ' ')

'Jadein-Ferrero vende excelente vivienda con reforma integral. Dispone de 2 habitaciones, salón de 20M2 con moderna cocina integrada y electrodomésticos a estrenar, impecable baño con plato de ducha, patio de 20M2, pintura lisa, A/A, escayola decorativa, suelos de mármol y trastero con acceso directo. Llame y concierte una visita sin compromiso.'

In [52]:
#This is to retrieve all the listing distribution details "Distribución".

all_listing_details = unique_soup.find_all('article', {'class': 'has-aside'})
distribution_details = all_listing_details[1].find_all('li')

all_distribution_details = []

for items in range(len(distribution_details)):
    print(distribution_details[items].text)
    all_distribution_details.append(distribution_details[items].text.strip().replace('\xa0', ' '))
    
all_distribution_details

2 habitaciones
Superficie 70 m2
1 Baño


['2 habitaciones', 'Superficie 70 m2', '1 Baño']

In [53]:
# THIS IS TO GET AL THE DETAILS LISTED IN THE BOX BELOW THE DESCRIPTION.

all_listings_distribution_details_list = []
all_listings_general_features_list = []
all_listings_community_features_list = []

#DISTRIBUTION MOST RELEVANT AND REPEATED ITEMS
dist_rooms = []
dist_meters = []
dist_baths = []
dist_trastero = []
dist_lavadero = []
dist_terraza = []


# GENERAL FEATURES: MOST RELEVANT OR REPEATED FEATURES. "CARACTERISTICAS PRINCIPALES"
feat_year_made = []
feat_unit_floor = []
feat_air_conditioner = []
feat_parking_spot = []
feat_heater = []

# COMMUNITY FEATURES: MOST RELEVANT OR REPEATED FEATURES. "EQUIPAMIENTO COMUNITARIO"

comm_elevator = []
comm_quota = []
comm_pool = []


all_listing_details = unique_soup.find_all('article', {'class': 'has-aside'})
distribution_details = all_listing_details[1].find_all('li')
general_features_details = all_listing_details[2].find_all('li')
community_equipment = all_listing_details[3].find_all('li')
    

one_listing_distribution_details_list= []
one_listing_all_features_details_list = []
one_listing_all_community_equipment_list = []

#TO EXTRACT DISTRIBUTION MOST RELEVANT AND REPEATED ITEMS TO LISTS.

if str('habitac') in all_listing_details[1].text:
    dist_rooms.append(all_listing_details[1].text[int(all_listing_details[1].text.find('habit'))-4:int(all_listing_details[1].text.find('habit'))].strip())
else:
    dist_rooms.append(np.NaN)

if str('Superficie') in all_listing_details[1].text:
    dist_meters.append(all_listing_details[1].text[int(all_listing_details[1].text.find('Superficie')) + len('Superficie'):int(all_listing_details[1].text.find('m2'))].strip())
else:
    dist_meters.append(np.NaN)

if str('Baño') in all_listing_details[1].text:
    dist_baths.append(all_listing_details[1].text[int(all_listing_details[1].text.find('Baño')) - 3:int(all_listing_details[1].text.find('Baño'))].strip())
else:
    dist_baths.append(np.NaN)

if str('Trastero') in all_listing_details[1].text:
    dist_trastero.append(True)
else:
    dist_trastero.append(np.NaN)

if str('Lavadero') in all_listing_details[1].text:
    dist_lavadero.append(True)
else:
    dist_lavadero.append(np.NaN)
    
if str('Terraza') in all_listing_details[1].text:
    dist_terraza.append(all_listing_details[1].text[int(all_listing_details[1].text.find('Terraza'))+len('Terraza'):int(all_listing_details[1].text.find('Terraza'))+len('Terraza') + 4].strip())
else:
    dist_terraza.append(np.NaN)

    
#Distribution    
for items in range(len(distribution_details)):
    print(distribution_details[items].text)
    one_listing_distribution_details_list.append(distribution_details[items].text.strip().replace('\xa0', ' '))
    
#General Features  
for items in range(len(general_features_details)):
    if len(general_features_details[items]) <= 2:
        print(general_features_details[items].text)
        one_listing_all_features_details_list.append(general_features_details[items].text.strip().replace('\xa0', ' '))
        
#Community Equipment  
for items in range(len(community_equipment)):
    if len(community_equipment[items]) <= 2:
        print(community_equipment[items].text)
        one_listing_all_community_equipment_list.append(community_equipment[items].text.strip().replace('\xa0', ' '))

    
#TO EXTRACT THE GENERAL FEATURES MOST RELEVANT AND REPEATED ITEMS TO LISTS.

if str('Calefacción') in all_listing_details[2].text:
    feat_heater.append(True)
elif str('Sin calefacción') in all_listing_details[2].text:
    feat_heater.append(False)
else:
    feat_heater.append(np.NaN)

if str('Aire acondicionado') in all_listing_details[2].text:
    feat_air_conditioner.append(True)
elif str('Sin aire acondicionado') in all_listing_details[2].text:
    feat_air_conditioner.append(False)
else:
    feat_air_conditioner.append(np.NaN)
    
if str('Plaza parking') in all_listing_details[2].text:
    feat_parking_spot.append(True)
elif str('Sin plaza parking') in all_listing_details[2].text:
    feat_parking_spot.append(False)
else:
    feat_parking_spot.append(np.NaN)

if str('Año construcción') in all_listing_details[2].text:
    feat_year_made.append(all_listing_details[2].text[int(all_listing_details[2].text.find('Año construcción')) + len('Año construcción'):int(all_listing_details[2].text.find('Año construcción')) + len('Año construcción') + 6].strip())
else:
    feat_year_made.append(np.NaN)

if str('Planta número') in all_listing_details[2].text:
    feat_unit_floor.append(all_listing_details[2].text[int(all_listing_details[2].text.find('Planta número')) + len('Planta número'):int(all_listing_details[2].text.find('Planta número')) + len('Planta número') + 3].strip())
else:
    feat_unit_floor.append(np.NaN)
    
    
#TO EXTRACT THE COMMUNITY MOST RELEVANT FEATURES AND REPEATED ITEMS TO LISTS.

if str('Ascensor') in all_listing_details[3].text:
    comm_elevator.append(True)
elif str('Sin ascensor') in all_listing_details[3].text:
    comm_elevator.append(False)
else:
    comm_elevator.append(np.NaN)
    
if str('Piscina comunitaria') in all_listing_details[3].text:
    comm_pool.append(True)
else:
    comm_pool.append(np.NaN)
    
if str('Cuota comunidad') in all_listing_details[3].text:
    comm_quota.append(all_listing_details[3].text[int(all_listing_details[3].text.find('Cuota comunidad')) + len('Cuota comunidad') :int(all_listing_details[3].text.find('€'))].strip())
else:
    comm_quota.append(np.NaN)
    

#Distribution
print(one_listing_distribution_details_list)
all_listings_distribution_details_list.append(one_listing_distribution_details_list)

#General Features        
print(one_listing_all_features_details_list)
all_listings_general_features_list.append(one_listing_all_features_details_list)

#Community Features        
print(one_listing_all_community_equipment_list)
all_listings_community_features_list.append(one_listing_all_community_equipment_list)

#THE FOLLOWING PART TO CREATE A LIST TO DOUBLE CHECK ITEMS.

print('\nCaracteristicas Distribución:\n')
print('dist_rooms', dist_rooms)
print('dist_meters', dist_meters)
print('dist_baths', dist_baths)
print('dist_trastero', dist_trastero)
print('dist_lavadero', dist_lavadero)
print('dist_terraza', dist_terraza)

print('\nCaracteristicas Generales:\n')
print('feat_year_made', feat_year_made)
print('feat_unit_floor', feat_unit_floor)
print('feat_air_conditioner', feat_air_conditioner)
print('feat_parking_spot', feat_parking_spot)
print('feat_heater', feat_heater)

print('\nEquipamiento Comunitario:\n')
print('comm_elevator', comm_elevator)
print('comm_pool', comm_pool)
print('comm_quota', comm_quota)


2 habitaciones
Superficie 70 m2
1 Baño
['2 habitaciones', 'Superficie 70 m2', '1 Baño']
[]
[]

Caracteristicas Distribución:

dist_rooms ['2']
dist_meters ['70']
dist_baths ['1']
dist_trastero [nan]
dist_lavadero [nan]
dist_terraza [nan]

Caracteristicas Generales:

feat_year_made [nan]
feat_unit_floor [nan]
feat_air_conditioner [nan]
feat_parking_spot [nan]
feat_heater [nan]

Equipamiento Comunitario:

comm_elevator [nan]
comm_pool [nan]
comm_quota [nan]


In [54]:
all_listing_details[3].text

'\nPregunta al anunciante sobre las características\n'

In [55]:
all_listing_details[3].text.find('€')

-1

In [56]:
len('Cuota comunidad')

15

In [57]:
all_listing_details[3].text[int(all_listing_details[3].text.find('Cuota comunidad')) + len('Cuota comunidad') :int(all_listing_details[3].text.find('€'))].strip()

'nunciante sobre las características'

In [58]:
all_listing_details = unique_soup.find_all('article', {'class': 'has-aside'})
all_features_details = all_listing_details[2].find_all('li')
#all_features_details[0]
#all_features_details[1]
#all_features_details[2]
#all_features_details[3]
str('Certificado energético') not in all_features_details[-1].text

False

In [59]:
#This is to retrieve all the listing features. "Caracteristicas Generales & Certificado Energetico"

#This for loop will get all the "caracteristicas principales"

all_features_list = []
consumo_ranking_list = []
emissions_ranking_list = []
consupmtion_kilowatts_list = []
emissions_kilograms_list = []


#This is where all the features are contained.
all_listing_details = unique_soup.find_all('article', {'class': 'has-aside'})
all_features_details = all_listing_details[2].find_all('li')


if all_listing_details[2].find('li', {'class': 'energy-container'}) == None:
        
    print('NO MENCIONA CERTIFICADO')
    rank_consumo = np.NaN
    rank_emisiones = np.NaN
    consupmtion_kilowatts = np.NaN
    emissions_kilograms = np.NaN

    consumo_ranking_list.append(rank_consumo)
    emissions_ranking_list.append(rank_emisiones)
    consupmtion_kilowatts_list.append(consupmtion_kilowatts)
    emissions_kilograms_list.append(emissions_kilograms)

    print('rank_consumo:', rank_consumo)
    print('rank_emisiones:', rank_emisiones)
    print('consupmtion_kilowatts:', consupmtion_kilowatts)
    print('emissions_kilograms:', emissions_kilograms)


for items in range(len(all_features_details)):
    
    if len(all_features_details[items]) == 1: # This part of the code gets all the unique listed features.
        print(all_features_details[items].text.strip())
        all_features_list.append(all_features_details[items].text.strip())
        
    elif len(all_features_details[items]) == 5: # This part of the code gets the energy related metrics if available.
    
        energy_certificate = all_features_details[-1].find_all('div', {'class': 'rating-box'})
        consumo = energy_certificate[0].text.replace('\r\n', '').strip().split()
        emisiones = energy_certificate[1].text.strip().split('\r\n')
        rank_consumo = consumo[1]
        rank_emisiones = emisiones[0][-1]
        consumo_ranking_list.append(rank_consumo)
        emissions_ranking_list.append(rank_emisiones)
        print(consumo)
        print(emisiones)
        print('rank_consumo:', rank_consumo)
        print('rank_emisiones:', rank_emisiones)

        try:
            consupmtion_kilowatts = int(consumo[2][:-2])
            consupmtion_kilowatts_list.append(consupmtion_kilowatts)
            print('consupmtion_kilowatts:', consupmtion_kilowatts)
        except:
            consupmtion_kilowatts = np.NaN
            consupmtion_kilowatts_list.append(consupmtion_kilowatts)
            print('consupmtion_kilowatts:', consupmtion_kilowatts)

        try:
            emissions_kilograms = int(emisiones[1][:-15]) 
            emissions_kilograms_list.append(emissions_kilograms)
            print('consupmtion_kilowatts:', consupmtion_kilowatts)
        except:
            cemissions_kilograms = np.NaN
            emissions_kilograms_list.append(emissions_kilograms)
            print('emissions_kilograms:', emissions_kilograms)

        
    else:
            print('CERTIFICADO EN TRAMITE')
            rank_consumo = np.NaN
            rank_emisiones = np.NaN
            consupmtion_kilowatts = np.NaN
            emissions_kilograms = np.NaN

            consumo_ranking_list.append(rank_consumo)
            emissions_ranking_list.append(rank_emisiones)
            consupmtion_kilowatts_list.append(consupmtion_kilowatts)
            emissions_kilograms_list.append(emissions_kilograms)

            print('rank_consumo:', rank_consumo)
            print('rank_emisiones:', rank_emisiones)
            print('consupmtion_kilowatts:', consupmtion_kilowatts)
            print('emissions_kilograms:', emissions_kilograms)

              
print(all_features_list)
print(consumo_ranking_list)
print(emissions_ranking_list)
print(consupmtion_kilowatts_list)
print(emissions_kilograms_list)

CERTIFICADO EN TRAMITE
rank_consumo: nan
rank_emisiones: nan
consupmtion_kilowatts: nan
emissions_kilograms: nan
[]
[nan]
[nan]
[nan]
[nan]


In [61]:
#This is to retrieve all the condo features, amenities or communal equipment.

all_listing_details = unique_soup.find_all('article', {'class': 'has-aside'})
equipamiento_comunitario = all_listing_details[3].find_all('li')


#This for loop will get all the "equipamiento comunitario"

for items in range(len(equipamiento_comunitario)):
    if len(equipamiento_comunitario[items]) == 1:
        print(equipamiento_comunitario[items].text.strip())

In [62]:
#This is to retrieve the listing id / unique number.

listing_id_section = unique_soup.find_all('section', {'class': 'options bg-white'})
listing_unique_id = listing_id_section[0].find_all('span', {'class': 'detail-id'})
listing_unique_id[0].text

listing_number = listing_unique_id[0].text[9:]
listing_number

'16424000000673'

In [63]:
#This is to retrieve the listing last modification or listing time-tag.

time_tag_item = unique_soup.find_all('p', {'class': 'time-tag'})
time_tag = time_tag_item[0].time.text
time_tag

'22/04/2021'

In [64]:
#This is to retrieve the district average price of the listing.
#Listing price and listing price per square meter is also available using this same tag/class.

listing_price_from_metrics = []
distric_price_from_metrics = []
square_meter_price_from_metric = []

price_metrics = unique_soup.find_all('div', {'class': 'round-block'})
price_metrics_details = price_metrics[0].find_all('div', {'class': 'bg-white round-100 f-left'})

try:
    listing_price_from_metrics.append(price_metrics_details[0].text[:int(price_metrics_details[0].text.find('€ Precio del anuncio'))].strip())
except:
    listing_price_from_metrics.append(np.NaN)
    
try:
    distric_price_from_metrics.append(price_metrics_details[1].text[:int(price_metrics_details[1].text.find('€/m2€/m2 distrito'))].strip())
except:
    distric_price_from_metrics.append(np.NaN)
    
try:
    square_meter_price_from_metric.append(price_metrics_details[2].text[:int(price_metrics_details[2].text.find('€/m2€/m2 del anuncio'))].strip())
except:
    square_meter_price_from_metric.append(np.NaN)


print()
print(listing_price_from_metrics)
print(distric_price_from_metrics)
print(square_meter_price_from_metric)


['290.000']
['5.284']
['4.143']


In [65]:
price_metrics_details[0].text

'290.000 € Precio del anuncio '

In [66]:
print(price_metrics)
distric_metrics = []
square_meter_metric = []

[<div class="round-block">
<div class="bg-white round-100 f-left">290.000 € <span>Precio del anuncio</span> </div>
<div class="bg-white round-100 f-left">5.284 €/m<sup>2</sup><span>€/m<sup>2</sup> distrito</span></div>
<div class="bg-white round-100 f-left">4.143 €/m<sup>2</sup><span>€/m<sup>2</sup> del anuncio</span></div>
<div class="comparation">
<div class="map-help">22% más barato que la media<span>*</span></div>
</div>
</div>]


In [67]:
price_metrics_details[0].text

'290.000 € Precio del anuncio '

In [68]:
price_metrics_details[0].text.find('€ Precio del anuncio')

8

In [69]:
price_metrics_details[0].text[:int(price_metrics_details[0].text.find('€ Precio del anuncio'))].strip()

'290.000'

In [70]:
price_metrics_details[1].text

'5.284 €/m2€/m2 distrito'

In [71]:
price_metrics_details[1].text.find('€/m2€/m2 distrito')

6

In [72]:
price_metrics_details[1].text[:int(price_metrics_details[1].text.find('€/m2€/m2 distrito'))].strip()

'5.284'

In [73]:
price_metrics_details[2].text

'4.143 €/m2€/m2 del anuncio'

In [74]:
price_metrics_details[2].text.find('€/m2€/m2 del anuncio')

6

In [75]:
price_metrics_details[2].text[:int(price_metrics_details[2].text.find('€/m2€/m2 del anuncio'))].strip()

'4.143'

In [76]:
#This is to retrieve the classic Google Map URL. (I couldn't! HAVE DOUBTS)

#https://www.codegrepper.com/code-examples/python/selenium+without+opening+browser+python
#https://phantomjs.org/quick-start.html
#https://sqa.stackexchange.com/questions/2609/running-webdriver-without-opening-actual-browser-window

map_section = unique_soup.find_all('section', {'class': 'map'})
map_image = map_section[0].find_all('div', {'class': 'map-image'})
map_zone_url = unique_soup.find_all('div', {'class': 'map-more'})
print('https:' + map_image[1].img.get('src')) # Esta es una foto de la zona
map_zone_url[0].a.get('href')

https://static11.habimg.com/imgh/16424-673/imgv2-gmaps-673_3542.gif


'https://www.habitaclia.com/comprar-piso-en-madrid-zona_embajadores_lavapies/provincia_madrid-madrid_capital-area_1/vivienda/vistamapa.htm'

In [77]:
len(all_listing_details[2].find_all('li'))

1

In [78]:
for i in all_features_details:
    print(len(i))

3


In [79]:
#Problematic URL

problematic_url = 'https://www.habitaclia.com/comprar-piso-magnifico_en_la_calle_orense_28020_orense_43_cuatro_caminos-madrid-i500003933609.htm?pag=651&f=&geo=p&from=list&lo=55'



In [83]:
problematic_url.startswith("https://www.habitaclia.com/comprar-piso-magnifico_en_la_calle_orense_28020_orense_43_cuatro_caminos-madrid-i500003933609", 0, 120) == True

True

In [81]:
len('https://www.habitaclia.com/comprar-piso-magnifico_en_la_calle_orense_28020_orense_43_cuatro_caminos-madrid-i500003933609')

120

In [131]:
import traceback
import sys

all_listings_features = []
all_listings_distribution_details = []
all_listings_equipamiento_comunitario = []

for link in tqdm(data[:950]):
    try:
        html = requests.get(link).content
        #print(link)
        unique_soup = BeautifulSoup(html, 'html.parser')
        
        #This is were all the "caracteristicas principales"
        all_listing_details = unique_soup.find_all('article', {'class': 'has-aside'})
        
        all_features_details = all_listing_details[2].find_all('li')

        for items in range(len(all_features_details)):
            if len(all_features_details[items]) <= 2 and all_features_details[items] not in all_listings_features:
                all_listings_features.append(all_features_details[items].text.strip())
            else:
                pass
            
        #This is were all the "distribution details are listed"
        distribution_details = all_listing_details[1].find_all('li')
        
        for items in range(len(distribution_details)):
            if len(distribution_details[items]) <= 3 and distribution_details[items] not in all_listings_distribution_details:
                all_listings_distribution_details.append(distribution_details[items].text.strip())
            else:
                pass
        
        #This is were all the "equipamiento comunitario is listed"
        equipamiento_comunitario = all_listing_details[3].find_all('li')
        
        for items in range(len(equipamiento_comunitario)):
            if len(equipamiento_comunitario[items]) <= 3 and equipamiento_comunitario[items] not in all_listings_equipamiento_comunitario:
                all_listings_equipamiento_comunitario.append(equipamiento_comunitario[items].text.strip())
            else:
                pass
        
    except Exception as e:
            print("Oops!", print(e), "occurred.")
            print("This happenned on link:", link)
            traceback.print_exc()
            print()
            traceback.print_exception(*sys.exc_info())
            print()
            pass

100%|██████████| 950/950 [09:39<00:00,  1.64it/s]


In [133]:
print('all_listings_features:', len(all_listings_features))
print('all_listings_distribution_details:', len(all_listings_distribution_details))
print('all_listings_equipamiento_comunitario:', len(all_listings_equipamiento_comunitario))

all_listings_features: 3023
all_listings_distribution_details: 3937
all_listings_equipamiento_comunitario: 1068


In [134]:
print(all_listings_features[0:50])

['Calefacción', 'Amueblado', 'Aire acondicionado', 'Calefacción', 'Sin plaza parking', 'Sin aire acondicionado', 'Planta número 1', 'Año construcción 1945', 'Calefacción', 'Sin plaza parking', 'Aire acondicionado', 'Planta número 1', 'Año construcción 1900', 'Calefacción', 'Plaza parking', 'Aire acondicionado', 'Planta número 2', 'Año construcción 2000', 'Calefacción', 'Sin plaza parking', 'Sin aire acondicionado', 'Año construcción 1964', 'Cerca de transporte público', 'Calefacción', 'Sin plaza parking', 'Aire acondicionado', 'Planta número 5', 'Año construcción 1960', 'Calefacción', 'Sin plaza parking', 'Aire acondicionado', 'Planta número 3', 'Año construcción 1989', 'Cerca de transporte público', 'Calefacción', 'Plaza parking', 'Amueblado', 'Aire acondicionado', 'Planta número 1', 'Año construcción 2020', 'Vistas a la montaña', 'Vistas a la ciudad', 'Cerca de transporte público', 'Calefacción', 'Sin plaza parking', 'Aire acondicionado', 'Planta número 1', 'Año construcción 1952', '

In [135]:
print(all_listings_distribution_details[0:50])

['4 habitaciones', 'Superficie 113\xa0m2', '3 Baños', 'Trastero', '6 habitaciones', 'Superficie 188\xa0m2', '2 Baños', 'Cocina tipo office: No', 'Estado cocina: buen estado', 'Trastero', '3 habitaciones', 'Superficie 105\xa0m2', '1 Baño', 'Cocina tipo office: No', 'Estado cocina: buen estado', 'Trastero', '2 habitaciones', 'Superficie 93\xa0m2', '2 Baños', 'Cocina tipo office: No', 'Estado cocina: buen estado', 'Trastero', '3 habitaciones', 'Superficie 134\xa0m2', '2 Baños', 'Cocina tipo office', 'Estado cocina: bueno', '2 habitaciones', 'Superficie 71\xa0m2', '2 Baños', 'Cocina tipo office: No', 'Estado cocina: bueno', '2 habitaciones', 'Superficie 70\xa0m2', '1 Baño', '3 habitaciones', 'Superficie 90\xa0m2', '1 Baño', 'Cocina tipo office', 'Estado cocina: bueno', '3 habitaciones', 'Superficie 126\xa0m2', '2 Baños', 'Despacho', 'Cocina tipo office: No', 'Estado cocina: obra nueva', 'Lavadero', 'Trastero', '3 habitaciones', 'Superficie 117\xa0m2']


In [136]:
print(all_listings_equipamiento_comunitario[0:50])

['Ascensor', 'Ascensor', 'Jardín comunitario', 'Vigilancia', 'Cuota comunidad 195€', 'Sin ascensor', 'Cuota comunidad 130€', 'Ascensor', 'Piscina comunitaria', 'Jardín comunitario', 'Vigilancia', 'Sin ascensor', 'Cuota comunidad 35€', 'Ascensor', 'Cuota comunidad 37€', 'Ascensor', 'Cuota comunidad 50€', 'Ascensor', 'Jardín comunitario', 'Vigilancia', 'Ascensor', 'Vigilancia', 'Cuota comunidad 125€', 'Ascensor', 'Ascensor', 'Vigilancia', 'Equipamiento deportivo', 'Sin ascensor', 'Cuota comunidad 38€', 'Ascensor', 'Jardín comunitario', 'Vigilancia', 'Cuota comunidad 120€', 'Ascensor', 'Ascensor', 'Ascensor', 'Ascensor', 'Piscina comunitaria', 'Jardín comunitario', 'Vigilancia', 'Cuota comunidad 110€', 'Equipamiento deportivo', 'Ascensor', 'Piscina comunitaria', 'Ascensor', 'Ascensor', 'Ascensor', 'Ascensor', 'Ascensor', 'Ascensor']


In [137]:
caracteristicas_generales_df = pd.DataFrame({'caracteristicas_generales': all_listings_features})

distribution_details_df = pd.DataFrame({'distribution_details': all_listings_distribution_details})

equipamiento_comunitario_df = pd.DataFrame({'equipamiento_comunitario': all_listings_equipamiento_comunitario})

In [259]:
caracteristicas_generales_df['caracteristicas_generales'].value_counts().head(25)

Calefacción                    646
Aire acondicionado             521
Plaza parking                  240
Sin aire acondicionado         168
Amueblado                      151
Cerca de transporte público    151
Sin plaza parking              146
Sin calefacción                 87
Planta número 2                 56
Vistas a la ciudad              49
Año construcción 1960           43
Planta número 1                 42
Año construcción 1970           35
Año construcción 1965           33
Año construcción 1900           31
Piscina propia                  31
Planta número 3                 26
Planta número 5                 17
Año construcción 1940           16
Planta número 4                 15
Año construcción 1968           15
Año construcción 1978           13
Vistas a la montaña             12
Año construcción 1969           12
Año construcción 1950           12
Name: caracteristicas_generales, dtype: int64

In [258]:
len(caracteristicas_generales_df['caracteristicas_generales'].unique())

131

In [257]:
distribution_details_df['distribution_details'].value_counts().head(20)

1 Baño                             427
Cocina tipo office                 357
2 Baños                            327
3 habitaciones                     305
Trastero                           268
2 habitaciones                     247
4 habitaciones                     158
1 habitación                       118
Lavadero                           117
3 Baños                            105
Cocina tipo office: No              85
1 Aseo                              74
5 habitaciones                      65
4 Baños                             41
Estado cocina: independiente        31
6 habitaciones                      24
5 Baños                             23
Superficie 65 m2                    19
Estado cocina: cocina amueblada     19
Superficie 72 m2                    16
Name: distribution_details, dtype: int64

In [156]:
distribution_details_df[distribution_details_df['distribution_details'].str.contains('Superficie')].count()

distribution_details    950
dtype: int64

In [256]:
len(distribution_details_df['distribution_details'].unique())

370

In [174]:
equipamiento_comunitario_df['equipamiento_comunitario'].value_counts().head(30)

Ascensor                  560
Piscina comunitaria       127
Sin ascensor               70
Jardín comunitario         63
Equipamiento deportivo     46
Vigilancia                 31
Cuota comunidad 50€        17
Cuota comunidad 40€         8
Cuota comunidad 60€         7
Cuota comunidad 70€         5
Cuota comunidad 35€         5
Cuota comunidad 125€        5
Cuota comunidad 45€         5
Cuota comunidad 150€        5
Cuota comunidad 200€        4
Cuota comunidad 130€        4
Cuota comunidad 38€         3
Cuota comunidad 170€        3
Cuota comunidad 25€         3
Cuota comunidad 110€        3
Cuota comunidad 90€         3
Cuota comunidad 120€        3
Cuota comunidad 42€         3
Cuota comunidad 115€        2
Cuota comunidad 400€        2
Cuota comunidad 350€        2
Cuota comunidad 30€         2
Cuota comunidad 177€        2
Cuota comunidad 57€         2
Cuota comunidad 81€         2
Name: equipamiento_comunitario, dtype: int64

In [260]:
equipamiento_comunitario_df['equipamiento_comunitario'].value_counts().tail(20)

Cuota comunidad 140€    1
Cuota comunidad 161€    1
Cuota comunidad 129€    1
Cuota comunidad 370€    1
Cuota comunidad 220€    1
Cuota comunidad 33€     1
Cuota comunidad 36€     1
Cuota comunidad 177€    1
Cuota comunidad 440€    1
Cuota comunidad 39€     1
Cuota comunidad 58€     1
Cuota comunidad 43€     1
Cuota comunidad 12€     1
Cuota comunidad 115€    1
Cuota comunidad 205€    1
Cuota comunidad 106€    1
Cuota comunidad 180€    1
Cuota comunidad 54€     1
Cuota comunidad 195€    1
Cuota comunidad 100€    1
Name: equipamiento_comunitario, dtype: int64

In [255]:
equipamiento_comunitario_df['equipamiento_comunitario'].unique()

array(['Ascensor', 'Jardín comunitario', 'Vigilancia',
       'Cuota comunidad 195€', 'Sin ascensor', 'Cuota comunidad 130€',
       'Piscina comunitaria', 'Cuota comunidad 35€',
       'Cuota comunidad 37€', 'Cuota comunidad 50€',
       'Cuota comunidad 125€', 'Equipamiento deportivo',
       'Cuota comunidad 38€', 'Cuota comunidad 120€',
       'Cuota comunidad 110€', 'Cuota comunidad 25€',
       'Cuota comunidad 88€', 'Cuota comunidad 91€',
       'Cuota comunidad 45€', 'Cuota comunidad 43€',
       'Cuota comunidad 52€', 'Cuota comunidad 200€',
       'Cuota comunidad 90€', 'Cuota comunidad 145€',
       'Cuota comunidad 65€', 'Cuota comunidad 73€',
       'Cuota comunidad 70€', 'Cuota comunidad 121€',
       'Cuota comunidad 140€', 'Cuota comunidad 100€',
       'Cuota comunidad 49€', 'Cuota comunidad 220€',
       'Cuota comunidad 12€', 'Cuota comunidad 66€',
       'Cuota comunidad 40€', 'Cuota comunidad 42€',
       'Cuota comunidad 150€', 'Cuota comunidad 350€',
       'Cuot

In [67]:
# #This is to retrieve all the listing features.

# all_listing_details = unique_soup.find_all('article', {'class': 'has-aside'})
# all_features_details = all_listing_details[2].find_all('li')

# #This for loop will get all the "caracteristicas principales"

# all_features_list = []
# consumo_ranking_list = []
# emissions_ranking_list = []
# consupmtion_kilowatts_list = []
# emissions_kilograms_list = []

# for items in range(len(all_features_details)):
#     if len(all_features_details[items]) == 1: # This part of the code gets all the unique listed features.
#         if str('Certificado energético') not in all_features_details[-1].text:
#             print(all_features_details[items].text.strip())
#         all_features_list.append(all_features_details[items].text.strip())
        
        
#     elif len(all_features_details[items]) == 5: # This part of the code gets the energy related metrics if available.
#         try:
#             energy_certificate = all_features_details[-1].find_all('div', {'class': 'rating-box'})
#             consumo = energy_certificate[0].text.replace('\r\n', '').strip().split()
#             emisiones = energy_certificate[1].text.strip().split('\r\n')
#             rank_consumo = consumo[1]
#             rank_emisiones = emisiones[0][-1]
#             consumo_ranking_list.append(rank_consumo)
#             emissions_ranking_list.append(rank_emisiones)
#             print(consumo)
#             print(emisiones)
#             print('rank_consumo', rank_consumo)
#             print('rank_emisiones', rank_emisiones)

#             consupmtion_kilowatts = int(consumo[2][:-2])
#             consupmtion_kilowatts_list.append(consupmtion_kilowatts)
#             print('consupmtion_kilowatts:', consupmtion_kilowatts)
            
#             if len(emisiones) == 1:
#                 emissions_kilograms = np.NaN
#             else:
#                 emissions_kilograms = int(emisiones[1][:-15])                       
            
#             emissions_kilograms_list.append(emissions_kilograms)
#             print('emissions_kilograms:', emissions_kilograms)
#         except:
#             rank_consumo = np.NaN
#             rank_emisiones = np.NaN
#             consupmtion_kilowatts = np.NaN
#             emissions_kilograms = np.NaN
            
#             consumo_ranking_list.append(rank_consumo)
#             emissions_ranking_list.append(rank_emisiones)
#             consupmtion_kilowatts_list.append(consupmtion_kilowatts)
#             emissions_kilograms_list.append(emissions_kilograms)
            
#             print('rank_consumo', rank_consumo)
#             print('rank_emisiones', rank_emisiones)
#             print('consupmtion_kilowatts:', consupmtion_kilowatts)
#             print('emissions_kilograms:', emissions_kilograms)
            
#     else:
#         if str('Certificado energético') not in all_features_details[-1].text:
#             rank_consumo = np.NaN
#             rank_emisiones = np.NaN
#             consupmtion_kilowatts = np.NaN
#             emissions_kilograms = np.NaN

#             consumo_ranking_list.append(rank_consumo)
#             emissions_ranking_list.append(rank_emisiones)
#             consupmtion_kilowatts_list.append(consupmtion_kilowatts)
#             emissions_kilograms_list.append(emissions_kilograms)

#             print('rank_consumo:', rank_consumo)
#             print('rank_emisiones:', rank_emisiones)
#             print('consupmtion_kilowatts:', consupmtion_kilowatts)
#             print('emissions_kilograms:', emissions_kilograms)
#         else:
#             rank_consumo = np.NaN
#             rank_emisiones = np.NaN
#             consupmtion_kilowatts = np.NaN
#             emissions_kilograms = np.NaN

#             consumo_ranking_list.append(rank_consumo)
#             emissions_ranking_list.append(rank_emisiones)
#             consupmtion_kilowatts_list.append(consupmtion_kilowatts)
#             emissions_kilograms_list.append(emissions_kilograms)

#             print('rank_consumo:', rank_consumo)
#             print('rank_emisiones:', rank_emisiones)
#             print('consupmtion_kilowatts:', consupmtion_kilowatts)
#             print('emissions_kilograms:', emissions_kilograms)
            
              
# print(all_features_list)
# print(consumo_ranking_list)
# print(emissions_ranking_list)
# print(consupmtion_kilowatts_list)
# print(emissions_kilograms_list)

In [ ]:
price_metrics_details[0]

In [ ]:
price_metrics_details[0].text

In [ ]:
price_metrics_details[1].text

In [ ]:
price_metrics_details[2].text

In [429]:
#Probando un URL especifico que me estoy trayendo del FULL SCRAPE

url_con_error_line112 = 'https://www.habitaclia.com/comprar-piso-magnifico_en_la_calle_orense_28020_orense_43_cuatro_caminos-madrid-i500003933609.htm?pag=651&f=&geo=p&from=list&lo=55'
html_con_error_line112 = requests.get(url_con_error_line112).content
soup_con_error_112 = BeautifulSoup(html_con_error_line112, 'html.parser')
all_listing_details_table_line112 = soup_con_error_112.find_all('section', {'class': 'detail'})
all_listing_details_zoom_error_line112 = all_listing_details_table_line112[0].find_all('div', {'class': 'max-1440 padding-y'})
all_features_details_zoom2_line112 = all_listing_details_table_line112[0].find_all('article', {'class': 'has-aside'})
#all_features_details_zoom2_line112[0]
all_features_details_zoom2_line112

[<article class="has-aside" id="js-translate">
 <h3 id="js-detail-description-title">Magnífico piso en la calle orense 28020 madrid</h3>
 <p class="detail-description" id="js-detail-description">Se vende magnífico piso situado en la calle Orense, en el barrio de Cuatro Caminos.<br/>El piso es exterior y se halla en una primera planta con ascensor. Su superficie es de 118 m2, de los cuales 100 m2 son útiles. Consta de cuatro habitaciones, un amplio salón comedor, cocina, baño, aseo y dos terrazas, una de las cuales tiene 18 m2 y da a la calle Orense.<br/>La vivienda está en muy buen estado de conservación, con las instalaciones de luz y agua recientemente reformadas, tiene suelos de tarima, aire acondicionado y ventanas de climalit.<br/>La finca tiene un espacioso portal, recientemente renovado, con portero físico y con acceso totalmente adaptado a personas con movilidad reducida.<br/>El inmueble está situado a cinco minutos andando de la estación de metro de Santiago Bernabéu, a diez m

In [421]:
url_para_evaluar = 'https://www.habitaclia.com/comprar-piso-fenomenal_arapiles-madrid-i15511003002180.htm?f=&geo=p&from=list&lo=55'
html_para_evaluar = requests.get(url_para_evaluar).content
soup_para_evaluar = BeautifulSoup(html_para_evaluar, 'html.parser')
all_listing_details_eval = soup_para_evaluar.find_all('section', {'class': 'detail'})
#all_listing_details_zoom_eval = all_listing_details_eval[0].find_all('div', {'class': 'max-1440 padding-y'})
all_features_details_zoom_eval = all_listing_details_eval[0].find_all('article', {'class': 'has-aside'})
#all_features_details_zoom2_line112[0]
all_features_details_zoom_eval[2].find_all('li')




[<li>Calefacción </li>,
 <li>Sin plaza parking</li>,
 <li>Aire acondicionado </li>,
 <li>Planta número 1</li>,
 <li>Año construcción 1952</li>,
 <li class="energy-container">
 <div class="energy-rating">
 <span>Certificado energético : </span>
 </div>
 <div class="detail-rating">
 <div class="rating-box">
                                     Consumo:
                                     <div class="rating c-F">
                                         F
                                     </div>
 322kW h m<sup>2</sup> / año                                
                             </div>
 <div class="rating-box">
                                 Emisiones: <div class="rating c-E">E</div>
 66kg CO<sub>2</sub> m<sup>2</sup> / año                            </div>
 <button class="energy-rating-info jq-tooltip-white button-transparent-orange" data-tooltip-content="#tooltip-energy">
                                      Ver etiqueta calificación energética
                              

In [377]:
len(all_listing_details_error_line112)

1

## Running WebDriver without opening actual browser window

https://sqa.stackexchange.com/questions/2609/running-webdriver-without-opening-actual-browser-window

https://www.codegrepper.com/code-examples/python/selenium+without+opening+browser+python


In [ ]:
# driver = webdriver.Chrome('/Users/manuelaquino/Bootcamp/final_project/chromedriver')
# driver.get(data[0])

In [ ]:
# selenium_map_image = driver.find_element_by_class_name('map-box')

# selenium_map_image

In [ ]:
# table = driver.find_element_by_tag_name('main[class="w-100 curve-top pointer-events-none gtmproductdetail"]')

# table

# rows = table.find_elements_by_tag_name('a')

# rows

In [ ]:
# driver.close()

In [ ]:
# map_section = unique_soup.find_all('section', {'class': 'map'})
# one_down_map = map_section[0].find_all('div', {'class': 'max-1440 padding-y'})
# two_down_map = one_down_map[0].find_all('article', {'class': 'has-aside f-left'})
# three_down_map = two_down_map[0].find_all('div', {'class': 'map-box'})
# four_down_map = three_down_map[0].find_all('div', {'class': 'map-image'})
# four_down_map

## Python Web Scraping - Dynamic Websites

https://docs.python-requests.org/projects/requests-html/en/latest/

https://www.tutorialspoint.com/python_web_scraping/python_web_scraping_dynamic_websites.htm

## Resources and Useful or interesting Links

https://www.geeksforgeeks.org/how-to-print-exception-stack-trace-in-python/

https://www.elmundo.es/economia/2014/02/21/5306681a22601d875e8b4581.html

https://www.educative.io/edpresso/what-is-the-none-keyword-in-python

https://www.programiz.com/python-programming/methods/built-in/reversed

https://www.w3schools.com/python/ref_string_find.asp

https://www.programiz.com/python-programming/methods/built-in/all

https://pynative.com/python-regex-capturing-groups/

https://tarifasgasluz.com/faq/certificado-eficiencia-energetica

https://www.ennaranja.com/hipotecas/elegir-bien/elige-bien-que-diferencia-hay-entre-una-vivienda-con-calificacion-energetica-a-y-g/